In [1]:
  
import h5py
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Activation, Concatenate
from tensorflow.keras.layers import Flatten, Dropout
from tensorflow.keras.layers import Convolution2D, MaxPooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D
import sys
sys.path.append("..") # Adds higher directory to python modules path.
import branchingdnn as branching
from branchingdnn.utils import *
from branchingdnn.dataset import prepare

In [3]:
dataset = prepare.dataset(tf.keras.datasets.cifar10.load_data(),32,5000,22500,(227,227), channel_first = True)

trainSize 45000
testSize 10000


In [3]:
tens = tf.constant((1,2,3))
tens

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([1, 2, 3])>

In [7]:
batch_size = 32
validation_size = 5000
shuffle_size = 22500
input_size=(227,227)
channel_first = False
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

#hack to get around the limitation of providing additional parameters to the map function for the datasets below 
def augment_images(image, label,input_size=input_size):
    if channel_first:
        #swap the channels around.
        image = tf.transpose(image, [2, 0, 1])
        
    return prepare.augment_images(image, label, input_size)

validation_images, validation_labels = train_images[:validation_size], train_labels[:validation_size] #get the first 5k training samples as validation set
train_images, train_labels = train_images[validation_size:], train_labels[validation_size:] # now remove the validation set from the training set.
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
validation_ds = tf.data.Dataset.from_tensor_slices((validation_images, validation_labels))



train_ds_size = len(list(train_ds))
test_ds_size = len(list(test_ds))
validation_ds_size = len(list(validation_ds))

print("trainSize {}".format(train_ds_size))
print("testSize {}".format(test_ds_size))
train_ds = (train_ds
                .map(augment_images)
                .shuffle(buffer_size=tf.cast(shuffle_size,'int64'))
                .batch(batch_size=batch_size, drop_remainder=True))

test_ds = (test_ds
                .map(augment_images)
                #   .shuffle(buffer_size=train_ds_size)
                .batch(batch_size=batch_size, drop_remainder=True))

validation_ds = (validation_ds
                .map(augment_images)
                #   .shuffle(buffer_size=validation_ds_size)
                .batch(batch_size=batch_size, drop_remainder=True))



trainSize 45000
testSize 10000
Tensor("args_0:0", shape=(32, 32, 3), dtype=uint8)
Tensor("transpose:0", shape=(3, 32, 32), dtype=uint8)
Tensor("args_0:0", shape=(32, 32, 3), dtype=uint8)
Tensor("transpose:0", shape=(3, 32, 32), dtype=uint8)
Tensor("args_0:0", shape=(32, 32, 3), dtype=uint8)
Tensor("transpose:0", shape=(3, 32, 32), dtype=uint8)


In [4]:


def SqueezeNet(nb_classes, inputs=(3, 224, 224)):
    """ Keras Implementation of SqueezeNet(arXiv 1602.07360)
    @param nb_classes: total number of final categories
    Arguments:
    inputs -- shape of the input images (channel, cols, rows)
    """

    input_img = Input(shape=inputs)
    conv1 = Convolution2D(
        96, (7, 7), activation='relu', kernel_initializer='glorot_uniform',
        strides=(2, 2), padding='same', name='conv1',
        data_format="channels_first")(input_img)
    maxpool1 = MaxPooling2D(
        pool_size=(3, 3), strides=(2, 2), name='maxpool1',
        data_format="channels_first")(conv1)
    fire2_squeeze = Convolution2D(
        16, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire2_squeeze',
        data_format="channels_first")(maxpool1)
    fire2_expand1 = Convolution2D(
        64, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire2_expand1',
        data_format="channels_first")(fire2_squeeze)
    fire2_expand2 = Convolution2D(
        64, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire2_expand2',
        data_format="channels_first")(fire2_squeeze)
    merge2 = Concatenate(axis=1)([fire2_expand1, fire2_expand2])

    fire3_squeeze = Convolution2D(
        16, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire3_squeeze',
        data_format="channels_first")(merge2)
    fire3_expand1 = Convolution2D(
        64, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire3_expand1',
        data_format="channels_first")(fire3_squeeze)
    fire3_expand2 = Convolution2D(
        64, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire3_expand2',
        data_format="channels_first")(fire3_squeeze)
    merge3 = Concatenate(axis=1)([fire3_expand1, fire3_expand2])

    fire4_squeeze = Convolution2D(
        32, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire4_squeeze',
        data_format="channels_first")(merge3)
    fire4_expand1 = Convolution2D(
        128, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire4_expand1',
        data_format="channels_first")(fire4_squeeze)
    fire4_expand2 = Convolution2D(
        128, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire4_expand2',
        data_format="channels_first")(fire4_squeeze)
    merge4 = Concatenate(axis=1)([fire4_expand1, fire4_expand2])
    maxpool4 = MaxPooling2D(
        pool_size=(3, 3), strides=(2, 2), name='maxpool4',
        data_format="channels_first")(merge4)

    fire5_squeeze = Convolution2D(
        32, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire5_squeeze',
        data_format="channels_first")(maxpool4)
    fire5_expand1 = Convolution2D(
        128, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire5_expand1',
        data_format="channels_first")(fire5_squeeze)
    fire5_expand2 = Convolution2D(
        128, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire5_expand2',
        data_format="channels_first")(fire5_squeeze)
    merge5 = Concatenate(axis=1)([fire5_expand1, fire5_expand2])

    fire6_squeeze = Convolution2D(
        48, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire6_squeeze',
        data_format="channels_first")(merge5)
    fire6_expand1 = Convolution2D(
        192, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire6_expand1',
        data_format="channels_first")(fire6_squeeze)
    fire6_expand2 = Convolution2D(
        192, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire6_expand2',
        data_format="channels_first")(fire6_squeeze)
    merge6 = Concatenate(axis=1)([fire6_expand1, fire6_expand2])

    fire7_squeeze = Convolution2D(
        48, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire7_squeeze',
        data_format="channels_first")(merge6)
    fire7_expand1 = Convolution2D(
        192, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire7_expand1',
        data_format="channels_first")(fire7_squeeze)
    fire7_expand2 = Convolution2D(
        192, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire7_expand2',
        data_format="channels_first")(fire7_squeeze)
    merge7 = Concatenate(axis=1)([fire7_expand1, fire7_expand2])

    fire8_squeeze = Convolution2D(
        64, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire8_squeeze',
        data_format="channels_first")(merge7)
    fire8_expand1 = Convolution2D(
        256, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire8_expand1',
        data_format="channels_first")(fire8_squeeze)
    fire8_expand2 = Convolution2D(
        256, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire8_expand2',
        data_format="channels_first")(fire8_squeeze)
    merge8 = Concatenate(axis=1)([fire8_expand1, fire8_expand2])

    maxpool8 = MaxPooling2D(
        pool_size=(3, 3), strides=(2, 2), name='maxpool8',
        data_format="channels_first")(merge8)
    fire9_squeeze = Convolution2D(
        64, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire9_squeeze',
        data_format="channels_first")(maxpool8)
    fire9_expand1 = Convolution2D(
        256, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire9_expand1',
        data_format="channels_first")(fire9_squeeze)
    fire9_expand2 = Convolution2D(
        256, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire9_expand2',
        data_format="channels_first")(fire9_squeeze)
    merge9 = Concatenate(axis=1)([fire9_expand1, fire9_expand2])

    fire9_dropout = Dropout(0.5, name='fire9_dropout')(merge9)
    conv10 = Convolution2D(
        nb_classes, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='valid', name='conv10',
        data_format="channels_first")(fire9_dropout)

    global_avgpool10 = GlobalAveragePooling2D(data_format='channels_first')(conv10)
    softmax = Activation("softmax", name='softmax')(global_avgpool10)

    return Model(inputs=input_img, outputs=softmax)

In [5]:

import h5py
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Activation, Concatenate
from tensorflow.keras.layers import Flatten, Dropout
from tensorflow.keras.layers import Convolution2D, MaxPooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D


def SqueezeNet(nb_classes, inputs=(227, 227,3)):
  """ Keras Implementation of SqueezeNet(arXiv 1602.07360)
  @param nb_classes: total number of final categories
  Arguments:
  inputs -- shape of the input images (channel, cols, rows)
  """

  input_img = Input(shape=inputs)
  conv1 = Convolution2D(
      96, (7, 7), activation='relu', kernel_initializer='glorot_uniform',
      strides=(2, 2), padding='same', name='conv1',
      data_format="channels_last")(input_img)
  maxpool1 = MaxPooling2D(
      pool_size=(3, 3), strides=(2, 2), name='maxpool1',
      data_format="channels_last")(conv1)
  fire2_squeeze = Convolution2D(
      16, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
      padding='same', name='fire2_squeeze',
      data_format="channels_last")(maxpool1)
  fire2_expand1 = Convolution2D(
      64, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
      padding='same', name='fire2_expand1',
      data_format="channels_last")(fire2_squeeze)
  fire2_expand2 = Convolution2D(
      64, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
      padding='same', name='fire2_expand2',
      data_format="channels_last")(fire2_squeeze)
  merge2 = Concatenate(axis=1)([fire2_expand1, fire2_expand2])

  fire3_squeeze = Convolution2D(
      16, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
      padding='same', name='fire3_squeeze',
      data_format="channels_last")(merge2)
  fire3_expand1 = Convolution2D(
      64, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
      padding='same', name='fire3_expand1',
      data_format="channels_last")(fire3_squeeze)
  fire3_expand2 = Convolution2D(
      64, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
      padding='same', name='fire3_expand2',
      data_format="channels_last")(fire3_squeeze)
  merge3 = Concatenate(axis=1)([fire3_expand1, fire3_expand2])

  fire4_squeeze = Convolution2D(
      32, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
      padding='same', name='fire4_squeeze',
      data_format="channels_last")(merge3)
  fire4_expand1 = Convolution2D(
      128, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
      padding='same', name='fire4_expand1',
      data_format="channels_last")(fire4_squeeze)
  fire4_expand2 = Convolution2D(
      128, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
      padding='same', name='fire4_expand2',
      data_format="channels_last")(fire4_squeeze)
  merge4 = Concatenate(axis=1)([fire4_expand1, fire4_expand2])
  maxpool4 = MaxPooling2D(
      pool_size=(3, 3), strides=(2, 2), name='maxpool4',
      data_format="channels_last")(merge4)

  fire5_squeeze = Convolution2D(
      32, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
      padding='same', name='fire5_squeeze',
      data_format="channels_last")(maxpool4)
  fire5_expand1 = Convolution2D(
      128, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
      padding='same', name='fire5_expand1',
      data_format="channels_last")(fire5_squeeze)
  fire5_expand2 = Convolution2D(
      128, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
      padding='same', name='fire5_expand2',
      data_format="channels_last")(fire5_squeeze)
  merge5 = Concatenate(axis=1)([fire5_expand1, fire5_expand2])

  fire6_squeeze = Convolution2D(
      48, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
      padding='same', name='fire6_squeeze',
      data_format="channels_last")(merge5)
  fire6_expand1 = Convolution2D(
      192, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
      padding='same', name='fire6_expand1',
      data_format="channels_last")(fire6_squeeze)
  fire6_expand2 = Convolution2D(
      192, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
      padding='same', name='fire6_expand2',
      data_format="channels_last")(fire6_squeeze)
  merge6 = Concatenate(axis=1)([fire6_expand1, fire6_expand2])

  fire7_squeeze = Convolution2D(
      48, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
      padding='same', name='fire7_squeeze',
      data_format="channels_last")(merge6)
  fire7_expand1 = Convolution2D(
      192, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
      padding='same', name='fire7_expand1',
      data_format="channels_last")(fire7_squeeze)
  fire7_expand2 = Convolution2D(
      192, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
      padding='same', name='fire7_expand2',
      data_format="channels_last")(fire7_squeeze)
  merge7 = Concatenate(axis=1)([fire7_expand1, fire7_expand2])

  fire8_squeeze = Convolution2D(
      64, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
      padding='same', name='fire8_squeeze',
      data_format="channels_last")(merge7)
  fire8_expand1 = Convolution2D(
      256, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
      padding='same', name='fire8_expand1',
      data_format="channels_last")(fire8_squeeze)
  fire8_expand2 = Convolution2D(
      256, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
      padding='same', name='fire8_expand2',
      data_format="channels_last")(fire8_squeeze)
  merge8 = Concatenate(axis=1)([fire8_expand1, fire8_expand2])

  maxpool8 = MaxPooling2D(
      pool_size=(3, 3), strides=(2, 2), name='maxpool8',
      data_format="channels_last")(merge8)
  fire9_squeeze = Convolution2D(
      64, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
      padding='same', name='fire9_squeeze',
      data_format="channels_last")(maxpool8)
  fire9_expand1 = Convolution2D(
      256, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
      padding='same', name='fire9_expand1',
      data_format="channels_last")(fire9_squeeze)
  fire9_expand2 = Convolution2D(
      256, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
      padding='same', name='fire9_expand2',
      data_format="channels_last")(fire9_squeeze)
  merge9 = Concatenate(axis=1)([fire9_expand1, fire9_expand2])

  fire9_dropout = Dropout(0.5, name='fire9_dropout')(merge9)
  conv10 = Convolution2D(
      nb_classes, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
      padding='valid', name='conv10'
      )(fire9_dropout)

  global_avgpool10 = GlobalAveragePooling2D()(conv10)
  softmax = Activation("softmax", name='softmax')(global_avgpool10)

  return Model(inputs=input_img, outputs=softmax)

In [6]:
model = SqueezeNet(10)
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 227, 227, 3) 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 114, 114, 96) 14208       input_1[0][0]                    
__________________________________________________________________________________________________
maxpool1 (MaxPooling2D)         (None, 56, 56, 96)   0           conv1[0][0]                      
__________________________________________________________________________________________________
fire2_squeeze (Conv2D)          (None, 56, 56, 16)   1552        maxpool1[0][0]                   
______________________________________________________________________________________________

In [7]:
train_ds, validation_ds, test_ds = dataset
model.compile(optimizer=tf.optimizers.SGD(lr=0.001), loss='SparseCategoricalCrossentropy', metrics=['accuracy',confidenceDifference],run_eagerly=True)


In [8]:
epocs = 1
model.fit(train_ds, epochs=epocs,validation_data=validation_ds,validation_freq=1)

ResourceExhaustedError: OOM when allocating tensor with shape[1,227,227,32] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[{{node resize/ResizeBilinear}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [18]:
inputs=(227, 227,3)
input_img = Input(shape=inputs)
conv1 = Convolution2D(
    96, (7, 7), activation='relu', kernel_initializer='glorot_uniform',
    strides=(2, 2), padding='same', name='conv1')(input_img)
maxpool1 = MaxPooling2D(
    pool_size=(3, 3), strides=(2, 2), name='maxpool1')(conv1)
fire2_squeeze = Convolution2D(
    16, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
    padding='same', name='fire2_squeeze')(maxpool1)
model = Model(inputs=input_img, outputs=fire2_squeeze)
model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 227, 227, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 114, 114, 96)      14208     
_________________________________________________________________
maxpool1 (MaxPooling2D)      (None, 56, 56, 96)        0         
_________________________________________________________________
fire2_squeeze (Conv2D)       (None, 56, 56, 16)        1552      
Total params: 15,760
Trainable params: 15,760
Non-trainable params: 0
_________________________________________________________________
